### Install & import


In [31]:
!pip install -U -q google-generativeai

In [32]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [33]:
from google.colab import userdata

# API_KEY=userdata.get('API_KEY') ## add ur key


In [34]:
#methods for calculating mrr and p@1
def calculate_mrr(predictions, answers):

    mrr_c = 0
    for i in range(len(predictions)):
        if answers[i] in predictions[i]:
            rank = 1 / (predictions[i].index(answers[i]) + 1)
            mrr_c += rank
    mrr = mrr_c / len(predictions)
    return mrr


def calculate_precision_at_1(predictions, answers):
    p_c = 0
    for i in range(len(predictions)):
        if answers[i] == predictions[i][0]:
            p_c += 1

    precision_at_1 = p_c / len(predictions)

    return precision_at_1



In [35]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [36]:
model = "gemini-pro" # @param {isTemplate: true}
contents_b64 = "W3sicm9sZSI6InVzZXIiLCAicGFydHMiIDogW3sidGV4dCI6ICJoZWxsbyJ9XX0sIHsicm9sZSI6ICJtb2RlbCIsICJwYXJ0cyI6IFt7InRleHQiOiAiSGVsbG8hIEhvdyBjYW4gSSBhc3Npc3QgeW91IHRvZGF5PyJ9XX1d" # @param {isTemplate: true}
generation_config_b64 = "e30=" # @param {isTemplate: true}
safety_settings_b64 = "e30=" # @param {isTemplate: true}
user_input_b64 = 'SG93IGRvZXMgZWxlY3RyaWNpdHkgd29yaz8=' #@param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [37]:
contents

[{'role': 'user', 'parts': [{'text': 'hello'}]},
 {'role': 'model', 'parts': [{'text': 'Hello! How can I assist you today?'}]}]

In [38]:
generation_config

{}

In [39]:
safety_settings

{}

In [40]:
processed_queries_path='/content/queriesProcessed.txt'



In [41]:
# parse the query responses from Lucene based Engine and create appropriate queries/batches fro LLM

queriesContents=open(processed_queries_path,'r+').read()
queries=queriesContents.split('|\n')


QueryToLLMPrefix='Can you re-rank the answers for the following queries generated by our engine?\n'
QueryToLLMPrefix+='Please only output a list of strings as text like: [str1, str2, str3], [str1, str2, str3]  \n'
queries_to_llm=[]
batches_len=20
oneFullQUery=''
oneFullQUery+=QueryToLLMPrefix
query_batches=[]
answers=[]
current_batch_text=''
current_batch_text+=QueryToLLMPrefix
for idx,q in enumerate(queries):

  QueryToLLM=''
  furtherContents=q.split('[content]\n')
  if len(furtherContents)!=4:
    continue
  query=furtherContents[1]
  answer=furtherContents[2]
  answersWatson=furtherContents[3]
  QueryToLLM+=QueryToLLMPrefix
  inside_Text=''
  inside_Text+=f'query: {query}\n'
  inside_Text+=f'Our list of answers ranked: {answersWatson}\n'
  QueryToLLM+=inside_Text
  queries_to_llm.append(QueryToLLM)
  answers.append(answer)
  oneFullQUery+=inside_Text
  current_batch_text+=inside_Text
  if (idx+1)%batches_len==0:
    query_batches.append(f'{current_batch_text}\n give code as a python list')
    current_batch_text=QueryToLLMPrefix





### Call the API

In [42]:
processed_queries=[]
query_responses_LLM_objects=[]
query_responses_LLM_strings=[]
oneFullQueryResponse=''

In [43]:
# The following code was meant to generate llm respones for created queries via api
# the api limit reached so we had to manually copy the responses from the engines



# gemini = genai.GenerativeModel(model_name=model)

# chat = gemini.start_chat(history=contents)


# for query in query_batches:
#   if query not in processed_queries:
#     response = chat.send_message(
#         query,
#         stream=stream)
#     query_responses_LLM_objects.append(response)
#     query_responses_LLM_strings.append(response.text)
#     processed_queries.append(query)


In [44]:
# queries in batches
for idx,q in enumerate(query_batches):
  print(f"Batch {idx+1}:\n",q)


Batch 1:
 Can you re-rank the answers for the following queries generated by our engine?
Please only output a list of strings as text like: [str1, str2, str3], [str1, str2, str3]  
query: CAPITAL CITY CHURCHES (Alex: We'll give you the church. You tell us the capital city in which it is located.) The High Kirk of St. Giles, where John Knox was minister
Our list of answers ranked: [Church of Scotland, John Knox, John the Baptist, Edinburgh, Perth, Scotland, Mary of Bethany, Alexander Kolchak, Annapolis, Maryland, Genius loci, Edessa]
query: NEWSPAPERS This Georgia paper is known as the AJC for short
Our list of answers ranked: [The Atlanta Journal-Constitution, Interstate 285, Georgia (U.S. state), Thoroughbred racing in Australia, Cherokee language, The Tabernacle, The Five Orange Pips, IBM Series/1, Red Skelton, William C. Roberts]
query: NAME THE PARENT COMPANY Milton Bradley games
Our list of answers ranked: [Axis & Allies, Hasbro, Vectrex, Battleship (game), Snakes and Ladders, Mic

In [45]:
# since openai chatgpt api costs money,
# we manually created 10 batches of queries and recorded the answers in following respective arrays

cgpt1='[Edinburgh, Scotland, Church of Scotland], [The Atlanta Journal-Constitution, Georgia (U.S. state), Interstate 285], [Hasbro, Battleship (game), Axis & Allies], [Helsinki, Finland, Lutheran Cathedral], [Ottoman Empire, Turkish Republic, History of Armenia], [Ohio, Taft Museum of Art, Museum of Science and Industry (Chicago)], [Otto von Bismarck, Wilhelm II, Tear down this wall!], [Richard Nixon, American National Exhibition, Leonid Brezhnev], [Casablanca, Spanish words, Morocco], [William Henry Harrison, The Log Cabin, Horace Greeley]'
cgpt2="[Cairo, Nile, Khartoum], [Six-Day War, Golan Heights, West Bank], [JAG, Mark Harmon, List of NCIS characters], [Arlington National Cemetery, USS Maine, Tomb of the Unknowns], [Knights of Columbus, Father Michael McGivney, Sacred Heart Catholic Church (McCartyville, Ohio)], [Three's Company, Don Knotts, Norman Fell], [Florida, Naples Museum of Art, Nativity scene], [New Mexico, Georgia O'Keeffe Museum, Alfred Stieglitz], [The Wall Street Journal, Daniel Hertzberg, James B. Stewart], [B'nai B'rith, Isaac Dittenhoefer, Cave of the Patriarchs]"
cgpt3="[Khmer language, History of Cambodia, Siem Reap], [Cycle rickshaw, Transport in Cambodia, Economy of Cambodia], [Michael Jackson, Michael Jackson, Man in the Mirror], [Anna Paquin, True Blood, Jimmy Harry], [The Help, Myrlie Evers-Williams, Myrlie Evers-Williams], [Hogan's Heroes, Stalag 13, John Carter (ER)], [Budapest, Matthias Church, Saint Matthias], [My Funny Valentine, Rodgers and Hart, Frank Sinatra], [Joe Tinker, Johnny Evers, Frank Chance], [Duce, Fascism in Europe, Benito Mussolini]"
cgpt4="[Aaron Burr, James R. Sharp, Richard Mentor Johnson], [Kangaroo, Marsupial, Koala], [Iberian Peninsula, Peninsular War, Napoleonic Wars], [San Francisco Chronicle, Mark Twain, Bret Harte], [Animal House, Otter, Flounder], [Mercedes-Benz Superdome, New Orleans, Saenger Theatre (New Orleans, Louisiana)], [Mattel, Little People, Hasbro], [Roe v. Wade, Supreme Court decision, N.Y. Times], [3M, Post-it notes, US Airways Group], [Idaho, The Sun Valley Center for the Arts, Edward Hopper]"
cgpt5="[Rob Reiner, UCLA, Meathead], [Kraft Foods, Jell-O, Jello mold], [Janet Jackson, Miss You Much, Fontana Records], [Madison, Wisconsin, USA, El Capitolio], [Cape Town, South Africa, Pretoria], [Philip IV of France, Jacques De Molay, Knights Templar], [Charles de Gaulle, France, World War II], [Rotary International, De Poelen, Dronrijp], [Komodo dragon, Indonesia, Conservation], [The Washington Post, Washington, D.C., USA]"
cgpt6="[Ouzo, Greece, Anise], [George Martin, Fifth Beatle, The Beatles], [Red Wolf, WWF, Bald Eagle], [Michigan, Kalamazoo Institute of Arts, Michael Bierut], [Richard E. Byrd, Bernt Balchen, South Pole], [Mezquita-Catedral, Mosque–Cathedral of Córdoba, Chichen Itza, Svetitskhoveli Cathedral], [Giant panda, Wolong Nature Reserve, China], [Alien and Sedition Acts, Kentucky and Virginia Resolutions, Acts of Congress], [Governor General of Canada, David Johnston, Canada], [Jackie Joyner-Kersee, UCLA, Heptathlon]"
cgpt7="[Monrovia, Liberia, United States, Jug Tavern], [Game Change, Julianne Moore, HBO], [Harrison Ford, Space, Jack Ryan], [Kelsey Grammer, Golden Globe, Boss], [Joan of Arc, Pierre Cauchon, Roman Catholic Diocese of Beauvais], [Boot Hill, Dodge City, Kansas], [Vladimir Putin, KGB, Russia], [Edna St. Vincent Millay, My Candle Burns at Both Ends], [Emancipation Proclamation, Slavery, United States], [Ben Affleck, Good Will Hunting, Jack Ryan]"
cgpt8="[John Keats, Lord Byron, William Wordsworth], [Richard Nixon, United States, Vice President of the United States], [The Making of the President 1960, John F. Kennedy, Theodore White], [Fontana Records, Father Figure, 1988], [Crest (toothpaste), Procter & Gamble, Toothpaste], [Confucius, Chinese philosopher, Zhang Binglin], [Michelle Obama, First Lady, Malia and Sasha's mom], [Horace, Axiologus, Greek, Worth], [Heather Locklear, Amanda, Melrose Place], [Bessie Coleman, Chicago airport, Street named in her honor]"
cgpt9="[Republic of China, Taiwan, Taiwan Relations Act], [Joker, One Chance (film), Warren Beatty], [Henry Wadsworth Longfellow, Paul Revere's Ride, Tales of a Wayside Inn], [Mayim Bialik, The Big Bang Theory, Amy Farrah Fowler], [The Salvation Army, Red kettle, Digital donations], [Nicolas Poussin, Adoration of the Golden Calf, Italian painter], [Ammonites, Amman, Mideast country], [Fontana Records, Rock With You, 1980], [James Dean, Rebel with a Cause, 1950 stage production], [The West Wing, 2001, President of the United States]"
cgpt10="[Jack Dempsey, Max Schmeling, Gene Tunney, Lou Stillman, Big band], [The Faerie Queene, John Keats, Edmund Spenser, Ode, English poetry], [Rosa Parks, Rita Dove, Beautiful Dreamer, Nellie Y. McKay, Louise Erdrich], [Souvlaki, Skewer, Awadhi cuisine, Serbian cuisine, Brazilian cuisine], [Henry Kissinger, Richard Nixon, William B. Saxbe, Zbigniew Brzezinski, John Connally], [Fontana Records, Beat It, Them Again, Judy Rodman, Funk metal], [France, History of Cambodia, Khmer Rouge, Majapahit, Travancore], [Sherlock Holmes, Robert Downey, Jr., Hans Zimmer, I. A. L. Diamond, Basil Rathbone], [Feta, Pickled cheese, Norwegian cuisine, English cuisine, Breakfast], [Alec Baldwin, 30 Rock (season 6), The Hunt for Red October, Harrison Ford, USS Dallas (SSN-700)]"

gpt_answers=[]

for i in range(10):
  eval(f'gpt_answers.append(cgpt{i+1})')

In [46]:
# parsing and adding cgpt results to appropriate lists
cgpt_answers=[]
for a in gpt_answers:
  answers_split=a.split('], ')

  for a_c in answers_split:
    c_list=[]
    for str_response_answer in a_c.split(', '):
      if str_response_answer.startswith('['):
        c_list.append(str_response_answer[1:])
      else:
        c_list.append(str_response_answer)
    cgpt_answers.append(c_list)


In [47]:
# storing gemini results manually

gemini1=[["Church of Scotland", "John Knox", "Edinburgh"],
["The Atlanta Journal-Constitution", "Georgia (U.S. state)", "Atlanta"],
["Hasbro", "Milton Bradley", "Axis & Allies"],
["Helsinki", "Turku", "Finland"],
["Ottoman Empire", "Turkish Republic", "Trabzon"],
["Cincinnati, Ohio", "Ohio", "Taft Museum of Art"],
["Otto von Bismarck", "Germany", "Wilhelm II"],
["Richard Nixon", "Nikita Khrushchev", "American National Exhibition"],
["Casablanca", "Morocco", "Spanish"],
["William Henry Harrison", "Whig party", "Horace Greeley"]
]
gemini2=[
["Cairo", "Nile", "Egypt"],
["Six-Day War", "Israel", "Golan Heights"],
["NCIS", "JAG", "Mark Harmon"],
["Arlington National Cemetery", "USS Maine", "Washington D.C."],
["Knights of Columbus", "Catholic", "Father Michael McGivney"],
["Three's Company", "Don Knotts", "Norman Fell"],
["Naples, Florida", "Florida", "Naples Museum of Art"],
["Santa Fe, New Mexico", "New Mexico", "Georgia O'Keeffe Museum"],
["The Wall Street Journal", "Daniel Hertzberg", "James B. Stewart"],
["B'nai B'rith", "Isaac Dittenhoefer", "Jewish"]
]

gemini3=[
["Khmer language", "Krama", "Cambodia"],
["Tuk-tuk", "Auto rickshaw", "Asia"],
["Michael Jackson", "Man In The Mirror", "1988"],
["Anna Paquin", "Sookie Stackhouse", "True Blood"],
["The Help", "Medgar Evers", "Myrlie Evers-Williams"],
["Hogan's Heroes", "Stalag 13", "Kinch"],
["Budapest", "Vienna", "Matthias Church"],
["Funny Girl", "Don't Rain on My Parade", "Barbra Streisand"],
["Tinker", "Evers & Chance", "Chicago Cubs"] ,
["Duce", "Benito Mussolini", "Italian"]
]
gemini4=[
["Aaron Burr", "James R. Sharp", "The Deadlocked Election of 1800"],
["Kangaroo", "Marsupial", "Australia"],
["Iberian Peninsula", "Peninsular War", "Spain"],
["San Francisco Chronicle", "Mark Twain", "Bret Harte"],
["Animal House", "Delta Tau Chi", "Comedy"],
["Mercedes-Benz Superdome", "New Orleans", "Louisiana"],
["Mattel", "Fisher-Price", "Toys"],
["Roe v. Wade", "Supreme Court", "New York Times"],
["3M", "Post-it notes", "Sticky notes"],
["Sun Valley, Idaho", "Idaho", "Sun Valley Center for the Arts"]
]
gemini5=[
["Rob Reiner", "Meathead", "UCLA"],
["Kraft Heinz", "Jell-O", "Food"],
["A-ha", "Miss You Much", "1989"],
["Monona, Wisconsin", "Granite dome", "Wisconsin"],
["Cape Town", "South Africa", "Port city"],
["Knights Templar", "Jacques De Molay", "Philip IV of France"],
["Charles de Gaulle", "France", "World War II"],
["Rotary International", "Gearwheel", "Emblem"],
["Komodo dragon", "Indonesia", "Lizard"],
["The Washington Post", "Washington D.C.", "Newspaper"]
]
gemini6=[
["Ouzo", "Greek liqueur", "Anise"],
["George Martin", "The Beatles", "Fifth Beatle"],
["World Wildlife Fund (WWF)", "Bald eagle", "Conservation"],
["Kalamazoo, Michigan", "Kalamazoo Institute of Arts", "Museum"],
["Richard E. Byrd", "South Pole", "Pilot"],
["Chichen Itza", "Sacred structure", "600s A.D."],
["Giant panda", "Wolong Nature Preserve", "China"],
["Alien and Sedition Acts", "Kentucky and Virginia Resolutions", "1798"],
["Governor General of Canada", "David Johnston", "Queen's representative"],
["Jackie Joyner-Kersee", "UCLA", "Heptathlon"]
]
gemini7=[
["Monrovia", "Liberia", "Wooden verandas"],
["Game Change", "John McCain", "Sarah Palin"],
["Harrison Ford", "Jack Ryan", "Patriot Games"],
["Tom Hanks", "Chicago", "Golden Globe"],
["Joan of Arc", "Pierre Cauchon", "Trial"],
["Boot Hill", "Dodge City, Kansas", "Alice Chambers"],
["Vladimir Putin", "KGB", "President of Russia"],
["Edna St. Vincent Millay", "My candle burns at both ends", "Sonnet 14"],
["Slavery", "National Freedom Day", "13th Amendment"],
["Ben Affleck", "The Sum of All Fears", "Good Will Hunting"]
]
gemini8=[
["Lord Byron", "John Keats", "Figurative"],
["Calvin Coolidge", "Sixth vice president", "President"],
["John F. Kennedy", "The Making of the President 1960", "Pulitzer Prize"],
["Michael Jackson", "Father Figure", "1988"],
["Procter & Gamble", "Crest toothpaste", "Parent company"],
["Confucius", "Exile", "400s B.C."],
["Michelle Obama", "Malia and Sasha's mom", "African-American women"],
["John Quincy Adams", "Axiologus", "Poet"],
["Heather Locklear", "UCLA", "Psychology major"],
["Chicago O'Hare International Airport", "Bessie Coleman", "Pilot"]
]
gemini9=[
["Taiwan", "Island", "Old Year's Resolutions"],
["Heath Ledger", "Joker", "Golden Globe"],
["Henry Wadsworth Longfellow", "Tales of a Wayside Inn", "Paul Revere"],
["Mayim Bialik", "Neuroscientist", "UCLA"],
["The Salvation Army", "Red kettle", "Charity"],
["Titian", "Adoration of the Golden Calf", "Italian painter"],
["Jordan", "Ammonites", "Mideast country"],
["Michael Jackson", "Rock With You", "1980"],
["James Dean", "UCLA", "Rebel"],
["Martin Sheen", "President of the United States", "Golden Globe"]
]
gemini10=[
["Jack Dempsey", "Manassa Mauler", "1927"],
["Epithalamion", "Edmund Spenser", "Keats"],
["Rosa Parks", "On the Bus with", "African-American women"],
["Souvlaki", "Skewer", "Greek food"],
["Henry Kissinger", "Nixon cabinet", "Germany"],
["Michael Jackson", "Beat It", "1983"],
["France", "Cambodia", "Royal Palace"],
["Robert Downey, Jr.", "Sherlock Holmes", "Golden Globe"],
["Feta", "Pickled cheese", "Greek food"],
["Alec Baldwin", "Jack Ryan", "30 Rock"]
]
gemini_answers_raw=[]
gemini_answers=[]
for i in range(10):
  eval(f'gemini_answers_raw.append(gemini{i+1})')


In [48]:
# parsing and adding gemini results to appropriate lists

for a_lists in gemini_answers_raw:
  for a_list in a_lists:
    gemini_answers.append(a_list)

In [49]:
# calculating final metrics values

mrr_cgpt = calculate_mrr(cgpt_answers, answers)
precision_at_1_cgpt = calculate_precision_at_1(cgpt_answers, answers)

print("MRR CGPT:", mrr_cgpt)
print("Precision@1 CGPT:", precision_at_1_cgpt)


mrr_gemini = calculate_mrr(gemini_answers, answers)
precision_at_1_gemini = calculate_precision_at_1(gemini_answers, answers)

print("MRR Gemini:", mrr_gemini)
print("Precision@1 Gemini:", precision_at_1_gemini)


MRR CGPT: 0.6683333333333334
Precision@1 CGPT: 0.65
MRR Gemini: 0.6633333333333333
Precision@1 Gemini: 0.64
